# Decorators

In computing, aspect-oriented programming (AOP) is a programming paradigm that aims to increase modularity by allowing the separation of cross-cutting concerns. It does so by adding behavior to existing code (an advice) without modifying the code, instead separately specifying which code is modified via a "pointcut" specification, such as "log all function calls when the function's name begins with 'set'". This allows behaviors that are not central to the business logic (such as logging) to be added to a program without cluttering the code of core functions.

## Example
Decorator provide a very useful method to add functionality to existing functions and classes.

Decorators are functions that wrap other functions or classes. Decorators use the @ syntax to “attach” a decorator to a function or class.

In [3]:
# Example
class C:
    def func():
        """No self here."""
        print('Method used as function.')

    func = staticmethod(func)

c = C()
c.func()

Method used as function.


In [2]:
class C:
    @staticmethod
    def func():
        """No self here."""
        print('Method used as function.')

c = C()
c.func()

Method used as function.


## Closures
We can use the concept of a closure for writing a function decorator.

In Python, a closure is typically a function defined inside another function. This inner function grabs the objects defined in its enclosing scope and associates them with the inner function object itself. The resulting combination is called a closure.

Closures are a common feature in functional programming languages. In Python, closures can be pretty useful because they allow you to create function-based decorators, which are powerful tools.

Using this technique, we can create a function
that takes a function as argument and returns a new function.



The closure allows to access arguments of the outer
function from within the inner function.



In [5]:
def outer(outer_arg):
    def inner(inner_arg):
        return inner_arg + outer_arg
    return inner

new_func = outer(10)

In [6]:
new_func

<function __main__.outer.<locals>.inner(inner_arg)>

In [7]:
new_func(7)

17

How does the new function have
access to this 10? It is stored in the attribute __closure__

In [8]:
new_func.__closure__[0].cell_contents

10

## Simple decorator

In [10]:
def hello(func):
    print('Hello')

@hello
def add(a, b):
    return a + b

Hello


In [11]:
add(10, 20)

TypeError: 'NoneType' object is not callable

In [12]:
# this is wrong because this is essentially what we have written
def add(a, b):
    return a + b
add = hello(add)

Hello


The correct way:

In [13]:
def hello(func):
    """Decorator function."""
    def call_func(*args, **kwargs):
        """Wrapper."""
        print('Hello')
        # Call original function and return its result.
        return func(*args, **kwargs)
    # Return function defined in this scope.
    return call_func

@hello
def add(a, b):
    return a + b

add(20, 300)

Hello


320

### Problems with docstrings

In [14]:
def add(a, b):
    """Add two objects."""
    return a + b

In [18]:
# accessing docstrings
add.__doc__

'Add two objects.'

In [19]:
@hello
def add(a, b):
    """Add two objects."""
    return a + b

add.__doc__

'Takes a arbitrary number of positional and keyword arguments.'

The docstring gets lost, best practice is to use functools wraps

In [21]:
import functools

def hello(func):
    @functools.wraps(func)
    def _hello(*args, **kwargs):
        """Wrapper."""
        print('Hello')
        # Call original function and return its result.
        return func(*args, **kwargs)
    # Return function defined in this scope.
    return _hello

@hello
def add(a, b):
    """Add two objects."""
    return a + b

add.__doc__

'Add two objects.'

### Parameterized Decorators

In [23]:
def say(text):
    def _say(func):
        @functools.wraps(func)
        def call_func(*args, **kwargs):
            """Wrapper"""
            print(text)
            return func(*args, **kwargs)
        return call_func
    return _say

@say('Hello')
def add(a, b):
    """Add two objects."""
    return a, b


add.__doc__


'Add two objects.'

In [24]:
add(10, 20)

Hello


(10, 20)

### Chaining decorators

In [25]:
@say('A')
@say('B')
@hello
def add(a, b):
    return a, b

add(10, 20)

A
B
Hello


(10, 20)

### Callable Instances
So far we used functions to write decorators. Actually, Python uses a callable. Often the terms function and callable
are used interchangeable. Any object that reacts to an added pair of parenthesis (callable_name()) is callable.
We can check if an object is callable:

In [26]:
from functools import wraps
class Say:
    def __init__(self, text):
        self.text = text

    def __call__(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print(self.text)
            return func(*args, **kwargs)
        return wrapper


@Say('Hello')
def add(a, b):
    return a + b

add(10, 20)

Hello


30